In [11]:
#TO DO: debug
import tensorflow as tf 
from tensorflow import keras
from keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda, Flatten, Activation
from keras import regularizers

In [12]:
def gconv(tensor, channels, groups): 
    input_ch = tensor.get_shape().as_list()[-1] 
    group_ch = input_ch // groups
    output_ch = channels // groups 
    groups_list = []

    for i in range(groups):
        group_tensor = tensor[:, :, :, i * group_ch: (i+1) * group_ch] 
        group_tensor = Conv2D(output_ch, 1)(group_tensor) 
        groups_list.append(group_tensor)
    
    output = Concatenate()(groups_list) 
    return output

In [13]:
def channel_shuffle(x, groups):
    _, width, height, channels = x.get_shape().as_list() 
    group_ch = channels // groups
    x = Reshape([width, height, group_ch, groups])(x) 
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

In [14]:
def bottleneck_block(tensor, expand=96, squeeze=16): 
    x = gconv(tensor, channels=expand, groups=4)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = channel_shuffle(x, groups=4)
    x = DepthwiseConv2D(kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = gconv(x, channels=squeeze, groups=4)
    x = BatchNormalization()(x)
    x = Add()([tensor, x]) 
    output = ReLU()(x) 
    return output

In [15]:
def getModel ():
    input = keras.Input(shape=(19, 19, planes), name='board')
    x = Conv2D(trunk, 1, padding='same', kernel_regularizer=regularizers.l2(0.0001))(input) 
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    for i in range (blocks):
        x = bottleneck_block (x, filters, trunk)
    
    policy_head = Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x) 
    policy_head = Flatten()(policy_head)
    policy_head = Activation('softmax', name='policy')(policy_head)
    
    value_head = GlobalAveragePooling2D()(x)
    value_head = Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
    value_head = Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
    model = keras.Model(inputs=input, outputs=[policy_head, value_head]) 
    
    return model

In [16]:
model = getModel()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data() 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss='categorical_crossentropy',metrics=['acc']) 
model.fit(x_train, y_train,batch_size=64, epochs=1, validation_split=0.2)

NameError: name 'planes' is not defined